# The Product Pricer Continued

A model that can estimate how much something costs, from its description.

## Fine Tune GPT Model

In [1]:
import os 
import re
import math 
import json 
import random 
from dotenv import load_dotenv
from huggingface_hub import login 
import matplotlib.pyplot as plt 
import numpy as np 
import pickle 
from collections import Counter
### LLM models 
from openai import OpenAI
from anthropic import Anthropic

### Intrnal Classes
from items import Item 
from testing import Tester 

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
### Environment 

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
HF_TOKEN = os.getenv("HF_TOKEN")

login(HF_TOKEN, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
%matplotlib inline

In [4]:
openai_client = OpenAI() 
claude_client = Anthropic()

In [5]:
### Load pickle files of data

with open("train.pkl", "rb") as f: 
    train = pickle.load(f)

with open("test.pkl", "rb") as f: 
    test = pickle.load(f)

In [6]:
### OpenAI recommends fine-tuning with populations of 50-100 examples
### But as the examples I am using is very small, so I will go with 200 examples (and 1 epoch)

fine_tune_train = train[:200]
fine_tune_validation = train[200:250]

## Step 1 -- Preparing Data

Prepare the data for fine-tuning in JSONL (JSON Lines) format and upload to OpenAI

In [7]:
def messages_for(item):
    system_message = "You estimate prices of items. Reply only with the price, no explanation"
    user_prompt = item.test_prompt().replace(" to the nearest dollar","").replace("\n\nPrice is $","")
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": f"Price is ${item.price:.2f}"}
    ]

In [8]:
# Convert the items into a list of json objects - a "jsonl" string
# Each row represents a message in the form:
# {"messages" : [{"role": "system", "content": "You estimate prices...

def make_jsonl(items): 
    result = ""
    for item in items: 
        messages = messages_for(item)
        msg_json = json.dumps(messages)
        result += '{"messages": '+ msg_json +'}\n'

    return result.strip()

In [9]:
### Write items to jsonl files

def write_jsonl(items, filename): 
    with open(filename, "w") as f: 
        jsonl = make_jsonl(items)
        f.write(jsonl)

In [10]:
write_jsonl(fine_tune_train, "fine_tune_train.jsonl")

In [11]:
write_jsonl(fine_tune_validation, "fine_tune_validation.jsonl")

In [12]:
### Upload the train file to OpenAI 

with open("fine_tune_train.jsonl", "rb") as f: 
    train_file = openai_client.files.create(file=f, purpose="fine-tune")

In [13]:
train_file

FileObject(id='file-JAMvTMcaZX9bseWNqVHXzF', bytes=184390, created_at=1762955282, filename='fine_tune_train.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)

In [14]:
### Upload the validation file to OpenAI 

with open("fine_tune_validation.jsonl", "rb") as f: 
    validation_file = openai_client.files.create(file=f, purpose="fine-tune")

In [15]:
validation_file

FileObject(id='file-XuZyznc7DyQAz9LAYawye6', bytes=45578, created_at=1762955284, filename='fine_tune_validation.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)

## Step 2 -- Fine Tune

First set up your weights & biases free account at:

https://wandb.ai

From the Avatar >> Settings menu, near the bottom, you can create an API key.

Then visit the OpenAI dashboard at:

https://platform.openai.com/account/organization

In the integrations section, you can add your Weights & Biases key.


In [16]:
wandb_integration = {"type": "wandb", "wandb": {"project": "gpt-pricer"}}

In [17]:
openai_client.fine_tuning.jobs.create(
    training_file=train_file.id, 
    model="gpt-4o-mini-2024-07-18", 
    seed=42, 
    hyperparameters={"n_epochs": 1}, 
    integrations= [wandb_integration], 
    suffix="pricer"
)

FineTuningJob(id='ftjob-7kCH0sM7ccF0P7Q7cLMKqhoC', created_at=1762955287, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=1), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-14kuOB4rpvyQ7RSuvSs1VXdL', result_files=[], seed=42, status='validating_files', trained_tokens=None, training_file='file-JAMvTMcaZX9bseWNqVHXzF', validation_file=None, estimated_finish=None, integrations=[FineTuningJobWandbIntegrationObject(type='wandb', wandb=FineTuningJobWandbIntegration(project='gpt-pricer', entity=None, name=None, tags=None, run_id='ftjob-7kCH0sM7ccF0P7Q7cLMKqhoC'))], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs=1))), user_provided_suffix='pricer', usage_metrics=None, s

In [23]:
openai_client.fine_tuning.jobs.list(limit=1)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-7kCH0sM7ccF0P7Q7cLMKqhoC', created_at=1762955287, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=1), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-14kuOB4rpvyQ7RSuvSs1VXdL', result_files=[], seed=42, status='running', trained_tokens=None, training_file='file-JAMvTMcaZX9bseWNqVHXzF', validation_file=None, estimated_finish=None, integrations=[FineTuningJobWandbIntegrationObject(type='wandb', wandb=FineTuningJobWandbIntegration(project='gpt-pricer', entity=None, name=None, tags=None, run_id='ftjob-7kCH0sM7ccF0P7Q7cLMKqhoC'))], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=1))), user_provided_suffix='pricer', usage_metr

In [32]:
job_id = openai_client.fine_tuning.jobs.list(limit=1).data[0].id
print(job_id)

ftjob-7kCH0sM7ccF0P7Q7cLMKqhoC


In [37]:
openai_client.fine_tuning.jobs.retrieve(job_id)

FineTuningJob(id='ftjob-7kCH0sM7ccF0P7Q7cLMKqhoC', created_at=1762955287, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:mighty-octopus:pricer:Cb5cvdzI', finished_at=1762955800, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=1), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-14kuOB4rpvyQ7RSuvSs1VXdL', result_files=['file-HjMRLGqDgTntjyiucCMCTp'], seed=42, status='succeeded', trained_tokens=39340, training_file='file-JAMvTMcaZX9bseWNqVHXzF', validation_file=None, estimated_finish=None, integrations=[FineTuningJobWandbIntegrationObject(type='wandb', wandb=FineTuningJobWandbIntegration(project='gpt-pricer', entity=None, name=None, tags=None, run_id='ftjob-7kCH0sM7ccF0P7Q7cLMKqhoC'))], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size=1, learning_rate_multiplier=1.8, 

In [38]:
openai_client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=10).data

[FineTuningJobEvent(id='ftevent-R443r74vQwkmxOKUgZv6OUO4', created_at=1762956629, level='info', message='The job has successfully completed', object='fine_tuning.job.event', data={}, type='message'),
 FineTuningJobEvent(id='ftevent-2lDhPFrtQaxMAreIsGWrE3HG', created_at=1762956624, level='info', message='Usage policy evaluations completed, model is now enabled for sampling', object='fine_tuning.job.event', data={}, type='message'),
 FineTuningJobEvent(id='ftevent-BU2Mbf4vjFfjn951fWOMdN59', created_at=1762956624, level='info', message='Moderation checks for snapshot ft:gpt-4o-mini-2024-07-18:mighty-octopus:pricer:Cb5cvdzI passed.', object='fine_tuning.job.event', data={'blocked': False, 'results': [{'flagged': False, 'category': 'harassment/threatening', 'enforcement': 'blocking'}, {'flagged': False, 'category': 'sexual', 'enforcement': 'blocking'}, {'flagged': False, 'category': 'sexual/minors', 'enforcement': 'blocking'}, {'flagged': False, 'category': 'propaganda', 'enforcement': 'blo